# Quickstart

This notebook shows how to use PySTAC to read through the public Sentinel catalog and write a local version.

### Reading STAC

First, we want to hook into PySTAC to allow for reading of HTTP STAC items, as described in [the STAC_IO Concepts docs](concepts.html#using-stac-io). 

__Note:__ this requires the [requests](https://requests.kennethreitz.org/en/master) library be installed.

In [1]:
from urllib.parse import urlparse
import requests
from pystac import STAC_IO

def requests_read_method(uri):
    parsed = urlparse(uri)
    if parsed.scheme.startswith('http'):
        return requests.get(uri).text
    else:
        return STAC_IO.default_read_text_method(uri)

STAC_IO.read_text_method = requests_read_method

We can then read the STAC catalog located at the publicly available endpoint hosted by AWS:

In [2]:
from pystac import Catalog

cat = Catalog.from_file('https://sentinel-stac.s3.amazonaws.com/catalog.json')

There are a lot of items in this catalog; crawling through it all would take a significant amount of time. Here, we lean on the fact that [link resolution is lazy](concepts.html#lazy-resolution-of-stac-objects) and get to a catalog that contains items:

In [3]:
while len(cat.get_item_links()) == 0:
    print('Crawling through {}'.format(cat))
    cat = next(cat.get_children())

Crawling through <Catalog id=sentinel-stac>
Crawling through <Collection id=sentinel-2-l1c>
Crawling through <Catalog id=9>
Crawling through <Catalog id=V>


We can print some information about the catalog, including how many children it has:

In [4]:
print(cat.description)
print('Contains {} items.'.format(len(cat.get_item_links())))

XK catalog
Contains 388 items.


Let's grab the first item., check out it's cloud cover, and start exploring the assets.

In [5]:
item = next(cat.get_items())

You can access [common metadata](https://github.com/radiantearth/stac-spec/blob/v0.9.0/item-spec/common-metadata.md) fields through the common_metadata property of the item:

In [6]:
item.common_metadata.platform

'sentinel-2b'

This particular stac item implements the [eo extension](https://github.com/radiantearth/stac-spec/tree/v0.9.0/extensions/eo) extension. We can access the extension information through the "ext" property that's part of every Catalog, Collection and Item. For instance, to get the cloud cover, we can use:

In [7]:
item.ext.eo.cloud_cover

41.52

we can see the cloud cover is in it's appropriate key in the Item's properties:

In [8]:
item.properties['eo:cloud_cover']

41.52

if we want to set the cloud cover, we can do that through the extension as well:

In [9]:
item.ext.eo.cloud_cover = 42.0
item.properties['eo:cloud_cover']

42.0

We can access the item's assets through the assets property, which is a dictionary:

In [10]:
for asset_key in item.assets:
    asset = item.assets[asset_key]
    print('{}: {} ({})'.format(asset_key, asset.href, asset.media_type))

thumbnail: https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/9/V/XK/2017/10/13/0/preview.jpg (None)
info: https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/9/V/XK/2017/10/13/0/tileInfo.json (None)
metadata: https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/9/V/XK/2017/10/13/0/metadata.xml (None)
tki: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/TKI.jp2 (image/jp2)
B01: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B01.jp2 (image/jp2)
B02: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B02.jp2 (image/jp2)
B03: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B03.jp2 (image/jp2)
B04: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B04.jp2 (image/jp2)
B05: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B05.jp2 (image/jp2)
B06: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B06.jp2 (image/jp2)
B07: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9

We can use the `to_dict()` method to convert an Asset, or any PySTAC object, into a dictionary:

In [11]:
asset = item.assets['B03']
asset.to_dict()

{'href': 'https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B03.jp2',
 'type': 'image/jp2',
 'title': 'Band 3 (green)',
 'eo:bands': [{'name': 'B03',
   'common_name': 'green',
   'gsd': 10.0,
   'center_wavelength': 0.56,
   'full_width_half_max': 0.045}]}

Here we use the eo extension to get the band information for the asset:

In [12]:
bands = item.ext.eo.get_bands(asset)
bands

[<Band name=B03>]

In [13]:
bands[0].to_dict()

{'name': 'B03',
 'common_name': 'green',
 'gsd': 10.0,
 'center_wavelength': 0.56,
 'full_width_half_max': 0.045}

### Writing a STAC

Let's walk the catalog again, but this time create local clones of the STAC object, so we can end up with a copy that we can save off to the local file system.

In [14]:
import itertools

cat = Catalog.from_file('https://sentinel-stac.s3.amazonaws.com/catalog.json')

# Setup the root of our local STAC
local_root = cat.clone()
local_root.clear_children()

# Loop over catalogs and clone
curr_local_cat = local_root
while len(cat.get_item_links()) == 0:
    print('Crawling through {}'.format(cat))
    cat = next(cat.get_children())
    local_cat = cat.clone()
    local_cat.clear_children()
    curr_local_cat.add_child(local_cat)
    curr_local_cat = local_cat
    
# Clear the items from the last local catalog
curr_local_cat.clear_children()
curr_local_cat.clear_items()

# Take the first 5 items
items = itertools.islice(cat.get_items(), 5)

# Clone and add them to our local catalog
curr_local_cat.add_items([i.clone() for i in items])

Crawling through <Catalog id=sentinel-stac>
Crawling through <Collection id=sentinel-2-l1c>
Crawling through <Catalog id=9>
Crawling through <Catalog id=V>


Now that we have a smaller STAC, let's map over the items to reduce it even further by only including the thumbnail assets in our items:

In [15]:
def item_mapper(item):
    thumbnail_asset = item.assets['thumbnail']
    
    new_assets = { 
        k:v 
        for k, v in item.assets.items()
        if k == 'thumbnail'
    }
    item.assets = new_assets
    return item

local_root_2 = local_root.map_items(item_mapper)
    

We can now normalize our catalog and save it somewhere local:

In [16]:
!mkdir -p ./quickstart_stac

In [17]:
local_root_2.normalize_hrefs('./quickstart_stac')

<Catalog id=sentinel-stac>

In [18]:
from pystac import CatalogType

local_root_2.save(catalog_type=CatalogType.SELF_CONTAINED)

In [19]:
local_root_2.describe()

* <Catalog id=sentinel-stac>
    * <Collection id=sentinel-2-l1c>
        * <Catalog id=9>
            * <Catalog id=V>
                * <Catalog id=XK>
                  * <Item id=S2B_9VXK_20171013_0>
                  * <Item id=S2A_9VXK_20171015_0>
                  * <Item id=S2B_9VXK_20171016_0>
                  * <Item id=S2B_9VXK_20171017_0>
                  * <Item id=S2A_9VXK_20171002_0>


In [20]:
for item in local_root_2.get_all_items():
    print('Item {}:')
    print('  Assets: {}'.format(item.assets))

Item {}:
  Assets: {'thumbnail': <Asset href=https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/9/V/XK/2017/10/13/0/preview.jpg>}
Item {}:
  Assets: {'thumbnail': <Asset href=https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/9/V/XK/2017/10/15/0/preview.jpg>}
Item {}:
  Assets: {'thumbnail': <Asset href=https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/9/V/XK/2017/10/16/0/preview.jpg>}
Item {}:
  Assets: {'thumbnail': <Asset href=https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/9/V/XK/2017/10/17/0/preview.jpg>}
Item {}:
  Assets: {'thumbnail': <Asset href=https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/9/V/XK/2017/10/2/0/preview.jpg>}


### Validating

If we have `jsonschema` installed, either manually or via installing PySTAC with the optional validation dependencies installed:

```
> pip install pystac[validation]
```

we can validate our STAC objects to ensure we didn't set any properties that break the spec:

In [21]:
for root, _, items in local_root_2.walk():
    root.validate()
    for item in items:
        item.validate()

If we have set something wrong, we'll get a `STACValidationException` when we validate:

In [22]:
item = next(local_root_2.get_all_items())
item.bbox = ['Not a valid bbox']
item.validate()

STACValidationError: Validation failed against schema at https://schemas.stacspec.org/v1.0.0-beta.2/item-spec/json-schema/item.json for STAC ITEM